In [1]:
import math
import torch
from torch import nn
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transforms, DataLoader, DataSet

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                          std=[0.229, 0.224, 0.225])
])

target_transform = transforms.Compose([
    transforms.Lambda(lambda y: torch.zeros(4, dtype=torch.float) \
      .scatter_(0, torch.tensor(y), value=1))
])

In [3]:
data_dir = '../data/images'
raw_data = datasets.ImageFolder(data_dir, 
                                transform=transform, 
                                target_transform=target_transform)
classes = raw_data.classes

In [4]:
sz = len(raw_data)
train_sz = math.floor(.8 * sz)
val_sz = sz - train_sz

In [5]:
train_dataset, val_dataset = random_split(raw_data, [train_sz, val_sz])

# Model Arch

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [7]:
classes = 4

# Define model
class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel, self).__init__()
        
        self.xfer = models.mobilenet_v3_small(pretrained=True)
        self.fc1 = nn.Linear(1000, classes)

    def forward(self, x):
        x = F.relu(self.xfer(x))
        return F.softmax(self.fc1(x), dim=1)
        
model = VisionModel().to(device)
print(model)

VisionModel(
  (xfer): MobileNetV3(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (relu): ReLU(inplace=True)
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1

# Training

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    current = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        current += len(X)
        print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}], {batch}")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
batch_size = 32
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(val_dataset, batch_size=batch_size)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.892317  [   32/  291], 0
loss: 1.281782  [   64/  291], 1
loss: 1.548884  [   96/  291], 2
loss: 2.271450  [  128/  291], 3
loss: 1.301106  [  160/  291], 4
loss: 3.129169  [  192/  291], 5
loss: 2.998152  [  224/  291], 6
loss: 2.245934  [  256/  291], 7
loss: 0.570473  [  288/  291], 8
loss: 1.131277  [  291/  291], 9
Test Error: 
 Accuracy: 50.7%, Avg loss: 0.546834 

Epoch 2
-------------------------------
loss: 0.982206  [   32/  291], 0
loss: 0.289445  [   64/  291], 1
loss: 0.223322  [   96/  291], 2
loss: 0.281471  [  128/  291], 3
loss: 0.185197  [  160/  291], 4
loss: 0.114093  [  192/  291], 5
loss: 0.177237  [  224/  291], 6
loss: 0.063245  [  256/  291], 7
loss: 0.108963  [  288/  291], 8
loss: 0.016269  [  291/  291], 9
Test Error: 
 Accuracy: 38.4%, Avg loss: 0.482279 

Epoch 3
-------------------------------
loss: 0.106550  [   32/  291], 0
loss: 0.094007  [   64/  291], 1
loss: 0.117993  [   96/  291], 2
loss: 0.111913  [